# Logistic Regression Customer Churn



A marketing agency has many customers that use their service to produce ads for the client/customer websites. They've noticed that they have quite a bit of churn in clients. They basically randomly assign account managers right now, but want you to create a machine learning model that will help predict which customers will churn (stop buying their service) so that they can correctly assign the customers most at risk to churn an account manager. Luckily they have some historical data, can you help them out? Create a classification algorithm that will help classify whether or not a customer churned. Then the company can test this against incoming data for future customers to predict which customers will churn and assign them an account manager.

The data is saved as customer_churn.csv. Here are the fields and their definitions:

    Name : Name of the latest contact at Company
    Age: Customer Age
    Total_Purchase: Total Ads Purchased
    Account_Manager: Binary 0=No manager, 1= Account manager assigned
    Years: Totaly Years as a customer
    Num_sites: Number of websites that use the service.
    Onboard_date: Date that the name of the latest contact was onboarded
    Location: Client HQ Address
    Company: Name of Client Company
    
Once you've created the model and evaluated it, test out the model on some new data (you can think of this almost like a hold-out set) that your client has provided, saved under new_customers.csv. The client wants to know which customers are most likely to churn given this data (they don't have the label yet).

In [ ]:
import findspark
findspark.init('/home/eissa/spark-2.3.1-bin-hadoop2.7')

In [1]:
from pyspark.sql import SparkSession

In [2]:
# Create Session
spark = SparkSession.builder.appName('logregconsult').getOrCreate()

In [4]:
# Load Data
data = spark.read.csv('customer_churn.csv',inferSchema=True,
                     header=True)

In [5]:
data.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Churn: integer (nullable = true)



In [6]:
data.describe().show()

+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+--------------------+--------------------+-------------------+
|summary|        Names|              Age|   Total_Purchase|   Account_Manager|            Years|         Num_Sites|            Location|             Company|              Churn|
+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+--------------------+--------------------+-------------------+
|  count|          900|              900|              900|               900|              900|               900|                 900|                 900|                900|
|   mean|         null|41.81666666666667|10062.82403333334|0.4811111111111111| 5.27315555555555| 8.587777777777777|                null|                null|0.16666666666666666|
| stddev|         null|6.127560416916251|2408.644531858096|0.4999208935073339|1.274449013194616|1.764835592035

In [7]:
data.columns

['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'Churn']

In [9]:
!pip install numpy

     |████████████████████████████████| 5.4 MB 19 kB/s  eta 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for numpy: filename=numpy-1.18.1-cp36-cp36m-linux_x86_64.whl size=13381811 sha256=9015af30809420226bd1689e01db1be41ea5f6c8e8a8d17382ef82385c2f23be
  Stored in directory: /root/.cache/pip/wheels/92/18/20/83339b2576b5911519a6b616d8b4a6df8b14358ba5cd612a0b
Successfully built numpy


In [10]:
from pyspark.ml.feature import VectorAssembler

In [11]:
# Create Feature Vector
assembler = VectorAssembler(inputCols=['Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites'],outputCol='features')

In [12]:
output = assembler.transform(data)
output.select('Features').show()

+--------------------+
|            Features|
+--------------------+
|[42.0,11066.8,0.0...|
|[41.0,11916.22,0....|
|[38.0,12884.75,0....|
|[42.0,8010.76,0.0...|
|[37.0,9191.58,0.0...|
|[48.0,10356.02,0....|
|[44.0,11331.58,1....|
|[32.0,9885.12,1.0...|
|[43.0,14062.6,1.0...|
|[40.0,8066.94,1.0...|
|[30.0,11575.37,1....|
|[45.0,8771.02,1.0...|
|[45.0,8988.67,1.0...|
|[40.0,8283.32,1.0...|
|[41.0,6569.87,1.0...|
|[38.0,10494.82,1....|
|[45.0,8213.41,1.0...|
|[43.0,11226.88,0....|
|[53.0,5515.09,0.0...|
|[46.0,8046.4,1.0,...|
+--------------------+
only showing top 20 rows



In [13]:
final_data = output.select('features','churn')

### Test Train Split

In [14]:
train_churn,test_churn = final_data.randomSplit([0.7,0.3])

### Fit the model

In [15]:
from pyspark.ml.classification import LogisticRegression

In [16]:
lr_churn = LogisticRegression(labelCol='churn')

In [17]:
fitted_churn_model = lr_churn.fit(train_churn)

In [18]:
training_sum = fitted_churn_model.summary
training_sum.predictions.show()

+--------------------+-----+--------------------+--------------------+----------+
|            features|churn|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|[22.0,11254.38,1....|  0.0|[4.42702588388791...|[0.98819113813869...|       0.0|
|[26.0,8939.61,0.0...|  0.0|[6.74142582917232...|[0.99882043092100...|       0.0|
|[28.0,9090.43,1.0...|  0.0|[1.48615989172726...|[0.81550119361629...|       0.0|
|[28.0,11128.95,1....|  0.0|[4.04933728447548...|[0.98286480908116...|       0.0|
|[28.0,11245.38,0....|  0.0|[4.07552961859397...|[0.98330039535383...|       0.0|
|[29.0,5900.78,1.0...|  0.0|[4.46359509933896...|[0.98861034862007...|       0.0|
|[29.0,8688.17,1.0...|  1.0|[2.77291746549093...|[0.94119466826719...|       0.0|
|[29.0,9378.24,0.0...|  0.0|[5.07662493726641...|[0.99379777065838...|       0.0|
|[29.0,9617.59,0.0...|  0.0|[4.79077765183687...|[0.99176242556087...|       0.0|
|[29.0,11274.46,

In [19]:
training_sum.predictions.describe().show()

+-------+-------------------+-------------------+
|summary|              churn|         prediction|
+-------+-------------------+-------------------+
|  count|                612|                612|
|   mean|0.16993464052287582|0.12745098039215685|
| stddev|0.37588258906480865|0.33375026074249314|
|    min|                0.0|                0.0|
|    max|                1.0|                1.0|
+-------+-------------------+-------------------+



### Evaluate results


In [20]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [21]:
pred_and_labels = fitted_churn_model.evaluate(test_churn)

In [22]:
pred_and_labels.predictions.show()

+--------------------+-----+--------------------+--------------------+----------+
|            features|churn|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|[25.0,9672.03,0.0...|    0|[4.99768031651118...|[0.99329171002819...|       0.0|
|[26.0,8787.39,1.0...|    1|[0.50711218760872...|[0.62412926022166...|       0.0|
|[27.0,8628.8,1.0,...|    0|[5.61090564109576...|[0.99635557517024...|       0.0|
|[28.0,8670.98,0.0...|    0|[8.19009959093042...|[0.99972269066408...|       0.0|
|[28.0,11204.23,0....|    0|[1.65928853730423...|[0.84014247453468...|       0.0|
|[29.0,10203.18,1....|    0|[3.79514026668229...|[0.97801447668596...|       0.0|
|[29.0,12711.15,0....|    0|[5.55197072810861...|[0.99613519103374...|       0.0|
|[29.0,13255.05,1....|    0|[3.82894404353012...|[0.97872970563468...|       0.0|
|[30.0,8874.83,0.0...|    0|[3.52977474004461...|[0.97152318095468...|       0.0|
|[30.0,10744.14,

### Using AUC

In [23]:
churn_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction',
                                           labelCol='churn')

In [24]:
auc = churn_eval.evaluate(pred_and_labels.predictions)

In [25]:
auc

0.7158641753503414

### Predict on brand new unlabeled data

In [26]:
final_lr_model = lr_churn.fit(final_data)

In [27]:
new_customers = spark.read.csv('new_customers.csv',inferSchema=True,
                              header=True)

In [28]:
new_customers.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)



In [29]:
test_new_customers = assembler.transform(new_customers)

In [30]:
test_new_customers.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- features: vector (nullable = true)



In [31]:
final_results = final_lr_model.transform(test_new_customers)

In [32]:
final_results.select('Company','prediction').show()

+----------------+----------+
|         Company|prediction|
+----------------+----------+
|        King Ltd|       0.0|
|   Cannon-Benson|       1.0|
|Barron-Robertson|       1.0|
|   Sexton-Golden|       1.0|
|        Wood LLC|       0.0|
|   Parks-Robbins|       1.0|
+----------------+----------+

